## 6.6 Using dynamic arrays

With dynamic arrays, we can now implement the unrestricted sequence ADT:
when the capacity is exhausted but an item is about to be inserted,
we resize the dynamic array to increase the capacity and then insert the item.
It has been shown that the best policy is to set the new capacity in proportion to the current capacity.
For example, in the implementation below
I set the new capacity to double of the current one.

Let's see why that's a good policy.
Resizing a full array, with length and capacity *n*, takes Θ(*n*),
in order to copy the *n* items to the larger static array.
Since the new array has capacity 2×*n*, we can insert *n* more items without resizing.
Spreading the cost of resizing over the subsequent insertions,
we see that the overhead per insertion is Θ(*n*) / *n* = Θ(1). In other words,
the run-time of a resize operation is the same as if
we added some constant time to each insertion.

Adding a constant time doesn't increase the complexity of an operation,
so the complexity of inserting in dynamic arrays is the same as for static arrays:
inserting at position *i* takes Θ(_n – i_) and appending (*i* = *n*–1) takes Θ(1).

Python's lists are implemented as dynamic arrays,
because this allows lists to grow as necessary, while
accessing, replacing and appending items in constant time.

<div class="alert alert-warning">
<strong>Note:</strong> Dynamic arrays are the best data structure for implementing the sequence ADT.
</div>

### 6.6.1 The `ArraySequence` class

To implement the sequence ADT with dynamic arrays,
we need the `Sequence` and `DynamicArray` classes.

In [1]:
%run -i ../m269_array
%run -i ../m269_sequence

The name and docstring of the new subclass reveal which data structure is used,
so that users know that indexing, replacing and appending take constant time.

In [2]:
# this code is also in m269_sequence.py

import math


class ArraySequence(Sequence):
    """A dynamic array implementation of the sequence ADT."""

    def __init__(self) -> None:
        """Create an empty sequence."""
        self.items = DynamicArray(1)
        self.size = 0

    def capacity(self) -> float:
        """Return how many items the sequence can hold: infinite."""
        return math.inf  # infinite capacity

    def length(self) -> int:
        """Return the number of items in the sequence.

        Postconditions: 0 <= self.length() <= self.capacity()
        """
        return self.size

    def get_item(self, index: int) -> object:
        """Return the item at position index.

        Preconditions: 0 <= index < self.length()
        Postconditions: the output is the n-th item of self, with n = index + 1
        """
        return self.items.get_item(index)

    def set_item(self, index: int, item: object) -> None:
        """Replace the item at position index with the given one.

        Preconditions: 0 <= index < self.length()
        Postconditions: post-self.get_item(index) == item
        """
        self.items.set_item(index, item)

    def insert(self, index: int, item: object) -> None:
        """Insert item at position index.

        Preconditions: 0 <= index <= self.length() < self.capacity()
        Postconditions: post-self is the sequence
        pre-self.get_item(0), ..., pre-self.get_item(index - 1),
        item, pre-self.get_item(index), ...,
        pre-self.get_item(pre-self.length() - 1)
        """
        if self.size == self.items.length():  # array full
            self.items.resize(2 * self.size)  # double the capacity

        for position in range(self.size - 1, index - 1, -1):
            self.items.set_item(position + 1, self.items.get_item(position))
        self.items.set_item(index, item)
        self.size = self.size + 1

The following code accesses the instance variables on purpose
to show how the internal static array evolves.
The array is printed with the `__str__` method inherited from `StaticArray`
and the sequence is printed with the `__str__` method inherited from `Sequence`.

In [3]:
sequence = ArraySequence()
print("array", sequence.items, "stores sequence", sequence)
for value in range(0, 5):
    sequence.append(value)
    print("array", sequence.items, "stores sequence", sequence)

array [None] stores sequence []
array [0] stores sequence [0]
array [0, 1] stores sequence [0, 1]
array [0, 1, 2, None] stores sequence [0, 1, 2]
array [0, 1, 2, 3] stores sequence [0, 1, 2, 3]
array [0, 1, 2, 3, 4, None, None, None] stores sequence [0, 1, 2, 3, 4]


As we can see, the length of the static array doubles step-wise from 1 to 8,
and the unused positions have value `None`.

Finally, let's test each method.

In [4]:
%run -i ../m269_test

test_init(ArraySequence())
for length in range(10):
    print("Testing length", length)
    test_append(ArraySequence(), length)
    test_insert_start(ArraySequence(), length)
    test_set_item(ArraySequence(), length)

Testing length 0
Testing length 1
Testing length 2
Testing length 3
Testing length 4
Testing length 5
Testing length 6
Testing length 7
Testing length 8
Testing length 9


#### Exercise 6.6.1 (optional)

Add a `remove` method to the `ArraySequence` class, then run these tests:

In [5]:
%run -i ../m269_test

for length in range(5):
    print("Testing length", length)
    test_remove(ArraySequence(), length)

After the tests pass, add code to your `remove` method to shrink the array
if the sequence, after the item was removed, is much shorter than the array.
It's up to you what 'much shorter' means.

You must shrink the capacity to a value that is proportional to the current one,
in order to keep the overhead per removal constant.
In addition, you must leave some spare capacity after shrinking:
otherwise the next `insert` would make the array grow again.
Having to grow a dynamic array right after shrinking it wouldn't be efficient.

[Hint](../31_Hints/Hints_06_6_01.ipynb)

⟵ [Previous section](06_5_dynamic_array.ipynb) | [Up](06-introduction.ipynb) | [Next section](06_7_linked_list.ipynb) ⟶